In [1]:
import pandas as pd
import numpy as np

In [2]:
eps_df = pd.read_csv("historical_eps.csv")
eps_df.set_index("Unnamed: 0", inplace=True)
rfr = pd.read_csv("CorpAAA.csv")
rfr.set_index("DATE", inplace=True)

In [3]:
def preceding_date1(given):
    given_date = pd.Timestamp(given)
    date_list = list(eps_df.index)
    date_list = pd.to_datetime(date_list)
    time_diffs = [(given_date - date).days for date in date_list]
    min_diff_index = min(i for i, diff in enumerate(time_diffs) if diff > 0)
    preceding_date = date_list[min_diff_index]
    return str(preceding_date.date())

In [4]:
def preceding_date2(given):
    given_date = pd.Timestamp(given)
    date_list = list(rfr.index)
    date_list = pd.to_datetime(date_list)
    time_diffs = [(given_date - date).days for date in date_list]
    min = 10
    min_ind = 0
    for i, diff in enumerate(time_diffs):
        if diff < min and diff >= 0:
            min_ind = i
    preceding_date = date_list[min_ind]
    return str(preceding_date.date())

In [5]:
def growth_rate(eps):
    eps_gr = ((eps[-1]/eps[0])**(1/len(eps)) - 1)*100
    return eps_gr

In [6]:
def stability(eps):
    mid_point = len(eps) // 2
    eps_mean_first_half = np.mean(eps[:mid_point])
    eps_mean_second_half = np.mean(eps[mid_point:])

    if (eps_mean_second_half > eps_mean_first_half) and (eps[-1]/eps[0])**(1/len(eps)) > 1:
        return 1
    else :
        return 0

In [15]:
def intrinsic_value(eps, bond_yield):
    if stability(eps):
        gr = growth_rate(eps)
        last_eps = eps[-1]
        return [(last_eps * (7 + 1 * gr) * 4.4) / bond_yield, (last_eps * (8 + 1.5 * gr) * 4.4) / bond_yield]
    else:
        return ("NA")

In [16]:
def calculate_intrinsic_value(given_date):
    
    q_date = preceding_date1(given_date)
    prev_eps = eps_df.loc[:q_date:-1]
    bond_yield = float(rfr.loc[preceding_date2(given_date)])

    results = []

    for i in eps_df.columns[1:]:
        results.append(intrinsic_value(prev_eps[i], bond_yield))

    healthy = []

    for i in results:
        if i == 'NA':
            healthy.append("NA")
            continue
        elif i[0] > 0 and i[1] > 0:
            healthy.append(i)
        else:
            healthy.append("NA")
    
    return healthy

In [20]:
IV = calculate_intrinsic_value("2018-09-11")

/var/folders/yl/5879yj014q11kbqcnjh_rd0m0000gn/T/ipykernel_22422/2779111783.py:6: RuntimeWarning: invalid value encountered in double_scalars
  if (eps_mean_second_half > eps_mean_first_half) and (eps[-1]/eps[0])**(1/len(eps)) > 1:


In [24]:
for i in range(len(eps_df.columns[1:])):
    if IV[i] != 'NA':
        print(eps_df.columns[i])

AAPL
ABT
ACN
ADBE
ADP
AMAT
AXP
BAC
BKNG
BLK
BRK-B
CAT
CB
CI
COP
CVX
DHR
DIS
ELV
ETN
GE
GOOG
GOOGL
GS
IBM
INTU
JNJ
JPM
LLY
LMT
LOW
LRCX
MA
MDLZ
MDT
META
MRK
MS
MSFT
MU
NEE
NOW
PEP
PG
PGR
QCOM
RTX
SBUX
SNPS
SPGI
SYK
T
TJX
TSLA
TXN
UNH
UNP
UPS
VRTX
